과제: 사용자 정의 층과 손실 함수를 사용한 신경망 만들기
목표
텐서플로와 케라스를 사용하여 사용자 정의 층과 손실 함수를 활용한 신경망을 설계하고 훈련합니다.

과제 요구사항
1. 데이터 준비

데이터셋: sklearn.datasets에서 제공하는 make_regression 데이터를 사용합니다.
데이터 처리:
훈련 데이터(80%) 와 테스트 데이터(20%) 로 분리하세요.
입력 데이터를 표준화하여 평균 0, 표준편차 1로 변환하세요.
2. 사용자 정의 층

MyDenseLayer라는 사용자 정의 층을 만드세요.
요구사항:
입력 크기에 따라 가중치(weight) 와 편향(bias) 를 생성하고 학습 가능하도록 설정합니다.
입력 데이터를 받아 ReLU 활성화 함수를 적용합니다.
3. 사용자 정의 손실 함수

후버 손실(Huber Loss) 를 구현하세요.
요구사항:
임계값(delta)은 1로 설정합니다.
작은 오차는 제곱 손실, 큰 오차는 선형 손실로 처리합니다.
4. 모델 설계 및 훈련

사용자 정의 층과 후버 손실 함수를 사용하여 신경망을 설계합니다.
요구사항:
구조: 2개의 은닉층(각 32개의 뉴런)과 1개의 출력층.
Optimizer: Adam.
평가지표: MSE (Mean Squared Error).
훈련: 10 epoch, batch size=32.
5. 평가 및 예측

요구사항:
테스트 데이터에서 MSE를 출력하세요.
테스트 데이터 중 첫 번째 샘플의 예측값과 실제값을 출력하세요.
출력
모델 훈련 중 손실값 출력:
테스트 데이터에서의 MSE 출력:
첫 번째 샘플의 예측값과 실제값 출력:

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. 데이터 준비
# 데이터 생성
data, target = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)
# 타겟 차원 변경
target = target.reshape(-1, 1)

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

# 데이터 표준화
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

# 2. 사용자 정의 층
class MyDenseLayer(Layer):
    def __init__(self, units):
        super(MyDenseLayer, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.weight = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
            name="weight"
        )
        self.bias = self.add_weight(
            shape=(self.units,),
            initializer="zeros",
            trainable=True,
            name="bias"
        )

    def call(self, inputs):
        return tf.nn.relu(tf.matmul(inputs, self.weight) + self.bias)

# 3. 사용자 정의 손실 함수
def huber_loss(y_true, y_pred, delta=1.0):
    error = tf.abs(y_true - y_pred)
    is_small_error = error <= delta
    small_error_loss = 0.5 * tf.square(error)
    big_error_loss = delta * error - 0.5 * tf.square(delta)
    return tf.where(is_small_error, small_error_loss, big_error_loss)

# 4. 모델 설계 및 훈련
model = Sequential([
    MyDenseLayer(32),
    MyDenseLayer(32),
    tf.keras.layers.Dense(1)  # 출력층
])

model.compile(optimizer=Adam(), loss=lambda y_true, y_pred: huber_loss(y_true, y_pred), metrics=["mse"])

# 모델 훈련
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# 손실값 출력
for epoch, loss in enumerate(history.history['loss'], 1):
    print(f"Epoch {epoch}: Loss = {loss}")

# 5. 평가 및 예측
# 테스트 데이터에서의 MSE
mse = model.evaluate(X_test, y_test, verbose=0)[1]
print(f"Test MSE: {mse}")

# 첫 번째 샘플의 예측값과 실제값
sample_pred = model.predict(X_test[:1])
actual_value = y_test[:1]

# 스케일링 복원
sample_pred = scaler_y.inverse_transform(sample_pred)
actual_value = scaler_y.inverse_transform(actual_value)

print(f"Predicted value: {sample_pred[0][0]}")
print(f"Actual value: {actual_value[0][0]}")

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.4422 - mse: 1.0666 - val_loss: 0.3399 - val_mse: 0.7759
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3797 - mse: 0.8685 - val_loss: 0.2688 - val_mse: 0.5891
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2659 - mse: 0.5829 - val_loss: 0.1181 - val_mse: 0.2400
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0962 - mse: 0.2014 - val_loss: 0.0164 - val_mse: 0.0327
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0164 - mse: 0.0329 - val_loss: 0.0089 - val_mse: 0.0177
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0072 - mse: 0.0143 - val_loss: 0.0056 - val_mse: 0.0111
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0071 - mse: 0.0143 - val_loss: 0.0030 - val_mse: 0.0059
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0037 - mse: 0.0074 - val_loss: 0.0024 - val_mse: 0.0048
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0028 - ms